In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms, models
from Models.selective_sequential import *
from Loss.triplet_regularized import *
from session import *
from LR_Schedule.lr_decay import LearningRateDecay
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
from session import LossMeter, EvalModel
from Layers.flatten import Flatten
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
import os, sys
from callbacks import TrainCallback

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/drake/anaconda3/envs/.

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

data = datasets.MNIST('/media/drake/MX500/Datasets/mnist/test', download=True, train=True, transform=transform)

In [3]:
class Identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def forward(self, x):
        return x

select = ['act1', 'act2', 'out']

def make_model(dropout=False, batchnorm=False):
    return SelectiveSequential(
    select,
    {'conv64': nn.Conv2d(1, 64, kernel_size=5, padding=2),
     'bn64': nn.BatchNorm2d(num_features=64) if batchnorm else Identity(),
     'act64': nn.ReLU(True),

     'max1': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-64': nn.Dropout(.05) if dropout else Identity(),  
    
     'conv192': nn.Conv2d(64, 192, kernel_size=5, padding=2),
     'bn192': nn.BatchNorm2d(num_features=192) if batchnorm else Identity(),
     'act192': nn.ReLU(True),   

     'max2': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-192': nn.Dropout(.1) if dropout else Identity(),
    
     'conv384': nn.Conv2d(192, 384, kernel_size=3, padding=1),
     'bn384': nn.BatchNorm2d(num_features=384) if batchnorm else Identity(),
     'act384': nn.ReLU(True),
     'drop-384': nn.Dropout(.15) if dropout else Identity(),
     
     'conv256a': nn.Conv2d(384, 256, kernel_size=3, padding=1),
     'bn256a': nn.BatchNorm2d(num_features=256) if batchnorm else Identity(),
     'act256a': nn.ReLU(True),
     'drop-256a': nn.Dropout(.1) if dropout else Identity(),
     
     'conv256b': nn.Conv2d(256, 256, kernel_size=3, padding=1),
     'bn256b': nn.BatchNorm2d(num_features=256) if batchnorm else Identity(),
     'act256b': nn.ReLU(True),

     'max3': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-256b': nn.Dropout(.1) if dropout else Identity(),
    
     'flatten': Flatten(),

     'fc1': nn.Linear(3 * 3 * 256, 512),
     'act1': nn.ReLU(True),
     'drop-fc1': nn.Dropout(.05) if dropout else Identity(),

     'fc2': nn.Linear(512, 512),
     'act2': nn.ReLU(True),
     'drop-fc2': nn.Dropout(.05) if dropout else Identity(),

     'out': nn.Linear(512, 10)})

In [4]:
class AccuracyValidator(TrainCallback):
    def __init__(self, val_data, accuracy_meter_fn):
        self.val_data = val_data
        self.val_accuracy_meter = accuracy_meter_fn()
        self.val_accuracies = []

    def run(self, session):
        self.val_accuracy_meter.reset()
        
        with EvalModel(session.model):
            for input, label, *_ in self.val_data:
                label = Variable(util.to_gpu(label))
                output = session.forward(input)
                
                self.val_accuracy_meter.update(output, label)
         
        accuracy = self.val_accuracy_meter.accuracy()
        
        self.val_accuracies.append(accuracy)  
        
    def on_epoch_end(self, session, lossMeter):        
        self.run(session) 

In [5]:
def train_sess(sess, trainloader, testloader):
    """
    Training helper function for consistent experiments
    """
    num_epochs = 30
    validator = AccuracyValidator(testloader, CustomOneHotAccuracy)
    lr_scheduler = LearningRateDecay(len(trainloader)*num_epochs, intervals=[25/30, 5/30], lrs=[.05, .01])
    schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
    sess.train(schedule, num_epochs)
    return np.max(validator.val_accuracies)

In [6]:
def k_fold(n, k):
    """
    Creates a list of ranges that partitions n samples into k folds
    """
    delt = n // k
    remainder = n % k
    ranges = []
    
    idx = 0
    
    for i in range(k):    
        prev_idx = idx
        
        idx += delt
        
        if remainder != 0:         
            remainder -= 1
            idx += 1
        
        val = np.arange(prev_idx, idx)
        train = np.setdiff1d(np.arange(0, n), val)
        
        ranges.append((train, val))
                               
    return ranges

In [7]:
params = [0, .05, .1, .15, .2, .25, .3, .35, .4]
folds = k_fold(len(data), 3)

In [8]:
accuracies = {}

for param in params:
    accs = []
       
    for idx, fold in enumerate(folds): 
        train, val = fold
        trainset = torch.utils.data.dataset.Subset(data, train)
        valset   = torch.utils.data.dataset.Subset(data, val)
        
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
        valloader   = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=False)
        
        model = make_model(dropout=False, batchnorm=False)
        criterion = TripletRegularizedMultiMarginLoss(param, .5, select, triplet_loss_fn=batch_all_triplet_loss)
        sess = Session(model, criterion, optim.SGD, 1e-4, log=False)
        
        acc = train_sess(sess, trainloader, valloader)
        
        accs.append(acc)
      
    accuracies[param] = {"mean": np.mean(accs), "folds": accs}
    
    df = pd.DataFrame(accuracies)
    df.to_csv("./MNIST-CV.csv")
        
    print(f"Lambda={param} Mean={round(np.mean(accs), 2)} ")
    
    for acc in accs:
        print(f"    {round(acc, 2)}")

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/torch/cuda/__init__.py:135: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))



Lambda=0 Mean=0.98 
    0.98
    0.98
    0.98



Lambda=0.05 Mean=0.99 
    1.0
    0.99
    0.99



Lambda=0.1 Mean=0.99 
    0.99
    0.99
    0.99



Lambda=0.15 Mean=0.99 
    0.99
    0.99
    0.99



Lambda=0.2 Mean=0.99 
    0.99
    0.99
    0.99



Lambda=0.25 Mean=0.99 
    0.99
    0.99
    0.99



Lambda=0.3 Mean=0.99 
    0.99
    0.99
    0.99



Lambda=0.35 Mean=0.99 
    0.99
    0.99
    0.99



Lambda=0.4 Mean=0.99 
    0.99
    0.99
    0.99


In [9]:
accuracies

{0: {'mean': 0.9839833333333333, 'folds': [0.98495, 0.9836, 0.9834]},
 0.05: {'mean': 0.99425, 'folds': [0.995, 0.9934, 0.99435]},
 0.1: {'mean': 0.9941333333333334, 'folds': [0.9949, 0.99375, 0.99375]},
 0.15: {'mean': 0.99375, 'folds': [0.9942, 0.9934, 0.99365]},
 0.2: {'mean': 0.9933666666666667, 'folds': [0.99365, 0.99315, 0.9933]},
 0.25: {'mean': 0.9935499999999999, 'folds': [0.9934, 0.9933, 0.99395]},
 0.3: {'mean': 0.9926166666666667, 'folds': [0.99355, 0.9923, 0.992]},
 0.35: {'mean': 0.9913333333333334, 'folds': [0.99235, 0.9918, 0.98985]},
 0.4: {'mean': 0.99035, 'folds': [0.9902, 0.9896, 0.99125]}}